In [1]:
## Packages need for data pre-process
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

from scipy import sparse
from collections import Counter

import nltk
from numpy import savetxt
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize

import itertools

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

# For evaluztion
from sklearn.metrics.cluster import normalized_mutual_info_score
from sklearn.metrics.cluster import adjusted_mutual_info_score
from sklearn.metrics.cluster import adjusted_rand_score

# Running time
from timeit import default_timer as timer

# Visualization
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from matplotlib.pyplot import figure

In [2]:
# Import preprocessed Dataset
four_after = pd.read_csv('/home/rep/scRNA-seq_clustering_to_Twitter/P1_preprocessing/doc_metadata_stemming_four_users.csv')
del four_after['Unnamed: 0']
True_Label = list(four_after["user_id_new"])

In [3]:
four_after

,user_id,user_id_new,screen_name,text,tweets_processed
0,27902825,2,UMichFootball,Leave it all on the field! @UMichFootball! Bes...,leav field umichfootbal best rivalri colleg fo...
1,27902825,2,UMichFootball,There’s no time to look backwards… only ahead!...,time look ahead watch umichfootbal ball goblu ...
2,27902825,2,UMichFootball,It’s called “The Game’ for a reason. \r\n\r\n#...,call game reason goblu beatosu
3,27902825,2,UMichFootball,"On Saturday, our seniors will play their final...",saturday senior play final game big hous senio...
4,27902825,2,UMichFootball,The Glasgow Decade with @UMichFootball is just...,glasgow decad umichfootbal complet stori goblu...
...,...,...,...,...,...
12502,19071682,3,breakingweather,A flash flood emergency is in effect for south...,flash flood emerg effect southwestern arkansa ...
12503,19071682,3,breakingweather,"Now that Barry, the first hurricane to make U....",barri first hurrican make landfal year come go...
12504,19071682,3,breakingweather,"Showers and locally heavy, drenching thunderst...",shower local heavi drench thunderstorm associ ...
12505,19071682,3,breakingweather,While Monday felt like a typical summer day in...,monday felt like typic summer day northeast te...


In [5]:
# read in Seurat clustering results 
Seurat = pd.read_csv("S_files/Seurat_estimated_clustering_four_PCA_25PCs.csv")
Seurat_results = pd.DataFrame(Seurat.drop(["Unnamed: 0", "Index"], axis =1))
col_name = Seurat_results.columns.to_list()

#### (1) Purity

In [10]:
# Purity
Correct_target = pd.DataFrame(four_after[["screen_name", "user_id_new"]])
Seurat_purity = []
for i, j in enumerate(col_name):
    n = pd.DataFrame(Seurat_results[j])
    df_compare = pd.concat([n, Correct_target], axis=1)
    numerator = df_compare.groupby([j, "user_id_new"], as_index=False)['screen_name'].count().sort_values('screen_name', ascending=False).drop_duplicates(j)["screen_name"].sum()
    purity = numerator/len(True_Label)
    Seurat_purity.append(purity)
    
Seurat_purity

[0.9337970736387623,
 0.9373950587670904,
 0.9394738946190133,
 0.942192372271528,
 0.9377948348924602,
 0.9344367154393539,
 0.9357959542656112,
 0.9375549692172384,
 0.9355560885903894,
 0.9393939393939394,
 0.9408331334452706,
 0.9380347005676821,
 0.9331574318381706,
 0.9360358199408332,
 0.9368353721915728,
 0.9387542975933477,
 0.9407531782201967,
 0.9413128647957144,
 0.9369952826417206,
 0.9365955065163508,
 0.9425121931718238,
 0.9439513872231551,
 0.9435516110977853,
 0.9436315663228592,
 0.9384344766930519,
 0.9360358199408332,
 0.9351563124650196,
 0.9376349244423123,
 0.9368353721915728,
 0.9369153274166466,
 0.9360358199408332,
 0.936275685616055,
 0.9353961781402415,
 0.9356360438154634,
 0.936195730390981,
 0.9358759094906852,
 0.93435676021428,
 0.9236427600543695,
 0.8972575357799633,
 0.8553609978412089,
 0.9122891180938675,
 0.911089789717758,
 0.6900935476133365]

#### (2) AMI

In [6]:
Seurat_AMI = []
for i, j in enumerate(col_name):
    n = Seurat_results[j]
    AMI = adjusted_mutual_info_score(True_Label, n)
    Seurat_AMI.append(AMI)
    
Seurat_AMI

[0.48398690062468613,
 0.5106351144926984,
 0.5196772631319925,
 0.5320067143421783,
 0.48382982586047546,
 0.4972895823272367,
 0.5060516642073369,
 0.5173266653903652,
 0.5286915918939059,
 0.5321838698911193,
 0.538274109645735,
 0.5411610806522236,
 0.5296532077594923,
 0.5364212114444717,
 0.5436683293146406,
 0.5392481040457957,
 0.5485247437095845,
 0.5643796367251295,
 0.5768908975052582,
 0.5806010466043888,
 0.5755349574782913,
 0.5998363835783872,
 0.5973683286983249,
 0.5853799932079866,
 0.6063675502538178,
 0.6049176614689397,
 0.6134970096101235,
 0.6249916753823287,
 0.6249022240687229,
 0.6246846985360589,
 0.6238910499502806,
 0.6252344031672528,
 0.6403138558419978,
 0.6440796593215309,
 0.6684389364650654,
 0.6630715123797833,
 0.692200677053879,
 0.7416558079139259,
 0.7186880539118575,
 0.678024064415196,
 0.759373038089491,
 0.7364289668304482,
 0.5922952520219171]

#### (3) ARI

In [7]:
Seurat_ARI = []
for i, j in enumerate(col_name):
    n = Seurat_results[j]
    ARI = adjusted_rand_score(True_Label, n)
    Seurat_ARI.append(ARI)
    
Seurat_ARI

[0.18856292664193555,
 0.25493791996790954,
 0.27540233820868404,
 0.30789118551523403,
 0.1882072465481079,
 0.22014195179710924,
 0.24189556139584142,
 0.258090859889454,
 0.28236624128176463,
 0.28373760095369394,
 0.29552212112729,
 0.3136516815658029,
 0.29083750450037754,
 0.29981953780438425,
 0.3179609278572711,
 0.2980424115994049,
 0.31402612249052353,
 0.34571249337280596,
 0.36588392830309396,
 0.3756144500419238,
 0.3881795768380742,
 0.42725693508183427,
 0.42229544650218115,
 0.3938249925712559,
 0.44847979791248,
 0.4448258799326736,
 0.46338366620030835,
 0.47657054983381436,
 0.47514939619204294,
 0.47271631007909554,
 0.4698931611167693,
 0.4705889961240594,
 0.49500660933958285,
 0.5059688829988775,
 0.5959258073244967,
 0.591919180881746,
 0.6464409608204807,
 0.7380663406720647,
 0.6981776132036639,
 0.6515977123726541,
 0.7785975562137205,
 0.7768170264083586,
 0.5520137834862557]

#### Summary

In [13]:
Seurat_evaluation_metrics = {'Purity': Seurat_purity, 'AMI': Seurat_AMI, 'ARI': Seurat_ARI}
Seurat_evaluation_metrics = pd.DataFrame(data = Seurat_evaluation_metrics)
Seurat_evaluation_metrics

,Purity,AMI,ARI
0,0.933797,0.483987,0.188563
1,0.937395,0.510635,0.254938
2,0.939474,0.519677,0.275402
3,0.942192,0.532007,0.307891
4,0.937795,0.483830,0.188207
5,0.934437,0.497290,0.220142
6,0.935796,0.506052,0.241896
7,0.937555,0.517327,0.258091
8,0.935556,0.528692,0.282366
9,0.939394,0.532184,0.283738


In [14]:
# Seurat_evaluation_metrics.to_csv("S_files/Seurat_evaluation_metrics_four_PCA_PCs25.csv")